# MAC-LRN Case Study
   ## Members:
   ### Caguiat, EJ
   ### Caro, Patricia
   ### Dienzo, Jericho Rafael
 
## Dataset: "Titanic: Machine Learning from Disaster"

 
 
 
 


# Introduction of the problem and dataset
### The training dataset that we are currently using describes different features for each passenger aboard the Titanic and shows whether or not they survived. The task is to correctly classify whether or not a given person with a given set of features (from the test dataset) would survive based on the training dataset.
#### The link to the kaggle dataset that we used: https://www.kaggle.com/c/titanic

# List of requirements
### The list of requirements that we used, and what we think are necessary in order to tackle this problem, are:
#### - Pandas (A Python library built on NumPy which is commonly used for machine learning problems)
#### - NumPy (A Python library used to process numerical data)
#### - The Kaggle Dataset (specifically the Titanic dataset) which can be obtained from the link provided above

### Import of necessary libraries and data sets

In [ ]:
import pandas as pd
import numpy as np
csvfile = pd.read_csv("/Users/Jericho/Documents/Subjects/MAC-LRN/all/train.csv")
print(csvfile)

# Data analysis
### The dataset that we used had many different features per person, all of which we deemed necessary in determining whether or not the person would live (more on this later). There were many anomalies in the dataset, such as null fields, fields that contained different datatypes than intended (int fields having chars), etc. Thus the data cleaning.